In [51]:
%load_ext autoreload
%autoreload 2

from collections import defaultdict

import dvu
import matplotlib.pyplot as plt
import pandas as pd
from os.path import join
import fitz
from tqdm import tqdm
import pathlib
import imodelsx.llm
import openai
openai.api_key = open('/home/chansingh/.OPENAI_KEY').read().strip()
plt.style.use('default')
dvu.set_style()

df = pd.read_csv('../data/main_updated.csv')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Extract text from pdfs

In [52]:
d = df[
    (df["found_paper (0=no, 1=yes, 2=low-qual)"] == 1)
    | (df["found_paper (0=no, 1=yes, 2=low-qual)"] == 2)
]
for i, row in tqdm(d.iterrows()):
    paper_file = join("../papers", str(row.id) + ".pdf")
    if pathlib.Path(paper_file).exists():
        with fitz.open(paper_file) as doc:  # open document
            text = chr(12).join([page.get_text() for page in doc])
            pathlib.Path(join("../papers", str(row.id) + ".txt")).write_bytes(
                text.encode()
            )

11it [00:00, 21.96it/s]


### Ask questions about the text

In [71]:
# llm = imodelsx.llm.get_llm("gpt-3.5-turbo-0613")
llm = imodelsx.llm.get_llm("gpt-4-0613")

In [72]:
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]
messages = [{"role": "user", "content": "What's the weather like in Boston?"}]

In [74]:
msg = llm(messages, functions=functions, return_str=False)

not cached
'choices'


In [75]:
print(msg.get('function_call'))

{
  "arguments": "{\n  \"location\": \"Boston\"\n}",
  "name": "get_current_weather"
}
